
# Recomendador


## Caminho do arquivo

In [0]:
%python

path_arquivo = 'dbfs:/FileStore/tables/musica/spotify/data-treated/data.parquet'


## Leitura do df (pandas)

In [0]:
%python

import pyspark.pandas as ps

df_data = ps.read_parquet(path_arquivo)

df_data.head()


## Informações do df

In [0]:
%python

df_data.info()


## Exclusão de registros nulos

In [0]:
%python

df_data = df_data.dropna()
df_data.info()


## Criação de coluna

In [0]:
%python

df_data['artists_song'] = df_data.artists + ' - ' + df_data.name
df_data.head()


## Informações do df

In [0]:
%python

df_data.info()


## Remoção de colunas

In [0]:
%python

x = df_data.columns.to_list()
x.remove('artists')
x.remove('id')
x.remove('name')
x.remove('artists_song')
x.remove('release_date')
x


## Tipo do dataframe atual

In [0]:
%python

type(df_data)


## Transformar o df de pandas para sql spark

In [0]:
%python

df_data = df_data.to_spark()
type(df_data)


## Tipo do dataframe atual

In [0]:
%python

type(df_data)


## Importação da feature de ML

In [0]:
%python

from pyspark.ml.feature import VectorAssembler


## Criação de um novo df

In [0]:
%python

dados_encoded_vector = VectorAssembler(
    inputCols=x,
    outputCol='features'
).transform(df_data)


## Visualizar novo df (feature)

In [0]:
%python

dados_encoded_vector.select('features').show(truncate=False, n=5)


## Importação da feature de ML

In [0]:
%python

from pyspark.ml.feature import StandardScaler


## Padronização dos dados

In [0]:
%python

scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaler = model_scaler.transform(dados_encoded_vector)


## Visualizar novo df (feature)

In [0]:
%python

dados_musicas_scaler.select('features_scaled').show(truncate=False, n=5)


## Qtd de features atual

In [0]:
%python

k = len(x)
k


## Importação da feature de ML

In [0]:
%python

from pyspark.ml.feature import PCA


## Redução do nº de features atual

In [0]:
%python

pca = PCA(k = k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca = model_pca.transform(dados_musicas_scaler)


## Representação atual das features

In [0]:
%python

sum(model_pca.explainedVariance) * 100


## Verificar até onde reduzir a feature

In [0]:
%python

lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores


## Importação da feature de ML

In [0]:
%python

import numpy as np


## Até onde podemos reduzir as features

In [0]:
%python

k = sum(np.array(lista_valores) <= 0.7)
k


## Redução de features (até 70%)

In [0]:
%python

pca = PCA(k = k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaler)


## Visualizar novo df (feature)

In [0]:
%python

dados_musicas_pca_final.select('pca_features').show(truncate=False, n=5)


## Importação da feature de ML

In [0]:
%python

from pyspark.ml import Pipeline


## Criação de pipeline

In [0]:
%python

pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=x, outputCol='features'), 
                        StandardScaler(inputCol='features',
outputCol='features_scaled'),
                        PCA(k=6, inputCol='features_scaled',
outputCol='pca_features')])


## Criando o dataframe c/ pipeline

In [0]:
%python

model_pca_pipeline = pca_pipeline.fit(df_data)


## Criando novo dataframe

In [0]:
%python

projection = model_pca_pipeline.transform(df_data)


## Visualizar novo df (feature)

In [0]:
%python

projection.select('pca_features').show(truncate=False, n=5)


## Importação da feature de ML

In [0]:
%python

from pyspark.ml.clustering import KMeans


##Criação de variável para (K-Means)

In [0]:
%python

SEED = 1224


## Utilização do K-Means

In [0]:
%python

kmeans = KMeans(k=50, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)


## Ajustes dos dados com fit

In [0]:
%python

modelo_kmeans = kmeans.fit(projection)


## Transformação dos dados

In [0]:
%python

projection_kmeans = modelo_kmeans.transform(projection)


## Visualizar novo df (feature)

In [0]:
%python

projection_kmeans.select(['pca_features', 'cluster_pca']).show()


## Importação da feature de ML

In [0]:
%python

from pyspark.ml.functions import vector_to_array


## Criação com K-Means

In [0]:
%python

projetion_kmeans = projection_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
   .withColumn('y', vector_to_array('pca_features')[1])


## Visualizar novo df (feature)

In [0]:
%python

projetion_kmeans.select('x', 'y', 'cluster_pca', 'artists_song').show()


## Importação da feature de ML

In [0]:
%python

import plotly.express as px


## Criação de visualização

In [0]:
%python

fig = px.scatter(projetion_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['artists_song'])
fig.show()


## Criação de variável (nome de música)

In [0]:
%python

nome_musica = 'Taylor Swift - Blank Space'


## Criação de um filtro (cluster)

In [0]:
%python

cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
cluster


## Criação de um filtro (musicas recomendadas)

In [0]:
%python

musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
    .select('artists_song', 'id', 'pca_features')
musicas_recomendadas.show()


## Criação de um filtro (componentes musica)

In [0]:
%python

componentes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
    .select('pca_features').collect()[0][0]
componentes_musica


## Importação da feature de ML

In [0]:
%python

from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f


## Distância euclidiana

In [0]:
%python

def calcula_distance(value):
    return euclidean(componentes_musica, value)

udf_calcula_distance = f.udf(calcula_distance, FloatType())

musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))


## Criação de um novo df

In [0]:
%python

recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

recomendadas.display()


## Função recomendador

In [0]:
%python

def recomendador(nome_musica):
    # Filtra o dataframe projetion_kmeans para obter o cluster da música com o nome especificado.
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]

    # Filtra o dataframe projetion_kmeans para obter todas as músicas que pertencem ao mesmo cluster.
    # Seleciona apenas as colunas artists_song, id e pca_features.
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                           .select('artists_song', 'id', 'pca_features')

    # Filtra o dataframe musicas_recomendadas para obter os componentes da música com o nome especificado.
    # Seleciona apenas a coluna pca_features e coleta o primeiro resultado.
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                              .select('pca_features').collect()[0][0]

    # Define uma função para calcular a distância euclidiana entre os componentes da música especificada e cada música recomendada.
    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    # Cria um User-Defined Function (UDF) a partir da função calcula_distance.
    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    # Adiciona uma nova coluna ao dataframe musicas_recomendadas contendo a distância entre cada música e a música especificada.
    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

    # Cria um novo dataframe contendo as 10 músicas mais próximas à música especificada, ordenadas por distância.
    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

    # Retorna o dataframe com as músicas recomendadas.
    return recomendadas


## Utilizando a função recomendador

In [0]:
%python

df_recomendada = recomendador('Taylor Swift - Blank Space')
df_recomendada.show()


## Instalação de biblioteca (spotipy)

In [0]:
!pip install spotipy


## Atualização de biblioteca (spotipy)

In [0]:
'''
pip install git+https://github.com/plamere/spotipy.git --upgrade
'''

# Tópico de resolução do StackOverflow: https://stackoverflow.com/questions/47028093/attributeerror-spotify-object-has-no-attribute-current-user-saved-tracks


## Importação da feature de ML

In [0]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


## Criação de autenticação

In [0]:
%python

scope = 'user-library-read playlist-modify-private'

OAuth = SpotifyOAuth(
    scope = scope,
    redirect_uri = 'http://localhost:5000/callback',
    client_id = '',
    client_secret = ''
)


## Criação de conexão

In [0]:
%python

client_credentials_manager = SpotifyClientCredentials(
    client_id = '',
    client_secret = ''
)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


## Criar variável (selecionar música)

In [0]:
%python

id = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('id').collect()[0][0]
id


## Selecionar música do spotify

In [0]:
%python

sp.track(id)


## Selecionador de ids

In [0]:
%python

playlist_id = df_recomendada.select('id').collect()


## Laço de seleção de ids

In [0]:
%python

playlist_track = []
for id in playlist_id:
    playlist_track.append(sp.track(id[0]))


## Função recomendador

In [0]:
%python

def recomendador(nome_musica):
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]
    
    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))
    
    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])
    
    id = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('id').collect()[0][0]
    
    
    playlist_id = recomendadas.select('id').collect()
    
    playlist_track = []
    
    for id in playlist_id:
        playlist_track.append(sp.track(id[0]))

    return len(playlist_track)


## Utilizando a função recomendador

In [0]:
%python

df_novo_recomendador = recomendador('Taylor Swift - Blank Space')
df_novo_recomendador


## Instalação de biblioteca (scikit-image)

In [0]:
!pip install scikit-image


## Exportar imagem do álbum

In [0]:
%python

import matplotlib.pyplot as plt
from skimage import io

nome_musica = 'Taylor Swift - Blank Space'

id = projetion_kmeans\
          .filter(projetion_kmeans.artists_song == nome_musica)\
          .select('id').collect()[0][0]

track = sp.track(id)

url = track["album"]["images"][1]["url"]
name = track["name"]

image = io.imread(url)
plt.imshow(image)
plt.xlabel(name, fontsize = 10)
plt.show()


## Exportar imagem do álbum (função)

In [0]:
%python

import matplotlib.pyplot as plt
from skimage import io

def visualize_songs(name,url):

    plt.figure(figsize=(15,10))
    columns = 5
    for i, u in enumerate(url):
        ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
        image = io.imread(u)
        plt.imshow(image)
        ax.get_yaxis().set_visible(False)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(name[i], fontsize = 10)
        plt.tight_layout(h_pad=0.7, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.grid(visible=None)
    plt.show()


## Laço de seleção de ids

In [0]:
%python

playlist_id = df_recomendada.select('id').collect()

name = []
url = []
for i in playlist_id:
    track = sp.track(i[0])
    url.append(track["album"]["images"][1]["url"])
    name.append(track["name"])


## Visualizar os álbuns

In [0]:
%python

visualize_songs(name,url)


## Função recomendador (final)

In [0]:
%python

def recomendador(nome_musica):
  # Calcula musicas recomendadas
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]

    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

  #Pegar informações da API

    playlist_id = recomendadas.select('id').collect()

    name = []
    url = []
    for i in playlist_id:
        track = sp.track(i[0])
        url.append(track["album"]["images"][1]["url"])
        name.append(track["name"])

  #Plotando capas 

    plt.figure(figsize=(15,10))
    columns = 5
    for i, u in enumerate(url):
        ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
        image = io.imread(u)
        plt.imshow(image)
        ax.get_yaxis().set_visible(False)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(name[i], fontsize = 10)
        plt.tight_layout(h_pad=0.7, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.grid(visible=None)
    plt.show()


## Utilizando a função recomendador (final)

In [0]:
%python

recomendador('Taylor Swift - Blank Space')